In [1]:
import torch
import numpy as np
import os

os.environ["CUDA_VISIBLE_DEVICES"]="0,1"

# Check System Devices

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

if torch.cuda.is_available():
    device_count = torch.cuda.device_count()
    for d in range(device_count):
        device_name = torch.cuda.get_device_name(d)
        print(device_name)
    #
#

device = torch.device('cuda:0')

Using device: cuda
NVIDIA GeForce GTX 1070
NVIDIA GeForce GTX 1070


# Autograd Example 1 
## - tensor.backward()

In [3]:
#Create tensors
x = torch.tensor(3., requires_grad=True, device=device)
w = torch.tensor(2., requires_grad=True, device=device)
b = torch.tensor(1., requires_grad=True, device=device)

# x = torch.tensor(1., requires_grad=True).to(device)
# w = torch.tensor(2., requires_grad=True).to(device)
# b = torch.tensor(3., requires_grad=True).to(device)

# Build a computational graph
y = w*x**2 + torch.sqrt(b)         # y = w*x^2 + sqrt(b)

# Compute gradients
y.backward()

# Print out the gradients
print(x.grad, (2*w*x).item())                            # dy/dx = 2*w*x
print(w.grad, (x**2).item())                             # dy/dw = x^2
print(b.grad, (0.5/torch.sqrt(b)).data.cpu().numpy())    # dy/db = (1/2)*(1/sqrt(b))


tensor(12., device='cuda:0') 12.0
tensor(9., device='cuda:0') 9.0
tensor(0.5000, device='cuda:0') 0.5


# Autograd Example 2: 
## - torch.autograd.grad() and high-order gradients

In [4]:
#Create tensors
x = torch.tensor(3., requires_grad=True, device=device)
w = torch.tensor(2., requires_grad=True, device=device)
b = torch.tensor(1., requires_grad=True, device=device)

# Build a computational graph
y = w*x**2 + torch.sqrt(b)         # y = w*x^2 + b

# Use autograd to compute the first-order gradient dy/dx
grad_x = torch.autograd.grad(y, x, create_graph=True)    # create_graph
print(grad_x, (2*w*x).item())

# Use autograd to compute the second-order gradient d^2y/dx^2
grad_xx = torch.autograd.grad(grad_x, x)
print(grad_xx, 2*w.item())



(tensor(12., device='cuda:0', grad_fn=<MulBackward0>),) 12.0
(tensor(4., device='cuda:0'),) 4.0
